You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# Technology Radar Analysis with Multi-Agent System

In this notebook, we'll create specialized agents to analyze and discuss the Technology Radar for different domains:

1. **Backend Technologies, Libraries, and Frameworks**
2. **Frontend Technologies, Libraries, and Frameworks**
3. **Testing Technologies, Libraries, and Frameworks**

Each agent will evaluate technologies based on adoption rates, industry usage trends, and licensing models (open-source vs. licensed).

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
import os
from utils import get_openai_api_key

os.environ["OPENAI_API_KEY"] = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

ImportError: cannot import name 'tool' from 'crewai.tools' (c:\Users\kiru79381\source\repos\techradar\venv\lib\site-packages\crewai\tools\__init__.py)

## Creating Specialized Technology Evaluator Agents

### Agent: Backend Expert

In [51]:
backend_expert = Agent(
    role="Backend Technology Expert",
    goal="Evaluate backend technologies, libraries, and frameworks from the Technology Radar.",
    backstory="You are an experienced backend architect with 15+ years of experience evaluating and implementing server-side technologies. "
              "You specialize in .NET and Python ecosystems and have deep knowledge of industry adoption trends, performance characteristics, "
              "and licensing models. You're known for providing practical insights about enterprise-level adoption decisions.",
    allow_delegation=False,
    max_iter=3,
    verbose=True
)

### Agent: Frontend Expert

In [52]:
frontend_expert = Agent(
    role="Frontend Technology Expert",
    goal="Evaluate frontend technologies, libraries, and frameworks from the Technology Radar.",
    backstory="You are a senior frontend architect who has led multiple enterprise UI projects. "
              "You specialize in Angular and modern web frameworks with expert knowledge on industry adoption, "
              "community support, and licensing implications. You excel at differentiating between hype and practical value.",
    allow_delegation=False,
    max_iter=3,
    verbose=True
)

### Agent: Testing Expert

In [53]:
testing_expert = Agent(
    role="Testing Technology Expert",
    goal="Evaluate testing technologies, libraries, and frameworks from the Technology Radar.",
    backstory="You are a quality assurance architect with expertise in test automation strategies across the entire tech stack. "
              "You have implemented testing frameworks for both frontend and backend systems and can evaluate tools based on "
              "industry adoption, effectiveness, learning curve, and licensing models. You advocate for practical testing approaches.",
    allow_delegation=False,
    max_iter=3,
    verbose=True
)

### Agent: Technology Radar Coordinator

In [54]:
coordinator = Agent(
    role="Technology Radar Coordinator",
    goal="Synthesize insights from domain experts and produce a final technology radar report.",
    backstory="You are a technology strategist responsible for maintaining and updating technology radars for the organization. "
              "You excel at synthesizing information from domain experts and can identify patterns, contradictions, and insights across domains. "
              "Your job is to ensure technology recommendations are consistent and aligned with business and technical objectives.",
    allow_delegation=True,
    max_iter=3,
    verbose=True
)

## Creating Specialized Tasks for Technology Evaluation

### Task: Backend Technology Evaluation

In [55]:
# First, let's load the technology radar content for use in our tasks
import pathlib

tech_radar_path = pathlib.Path('technology-radar.md')
tech_radar_content = tech_radar_path.read_text()

backend_eval_task = Task(
    description=(
        "Evaluate backend technologies, libraries, and frameworks from the provided Technology Radar document. "
        "Focus on .NET and Python ecosystems mentioned in the document. For each technology:"
        "\n1. Evaluate current adoption levels in the industry"
        "\n2. Specify whether it's open-source or licensed"
        "\n3. Provide insights on why certain technologies are in their respective quadrants (Adopt, Trial, Assess, Hold)"
        "\n4. Highlight any technologies that should be moved to different quadrants based on current industry trends"
    ),
    context=[{
        "description": "Technology Radar content for backend evaluation",
        "expected_output": "A detailed analysis of backend technologies",
        "technology_radar": tech_radar_content
    }],  # Context is now passed as a list of dictionaries with required fields
    expected_output="A comprehensive analysis of backend technologies with focus on adoption trends, licensing models, and recommendations for quadrant changes.",
    agent=backend_expert,
)

### Task: Frontend Technology Evaluation

In [56]:
frontend_eval_task = Task(
    description=(
        "Evaluate frontend technologies, libraries, and frameworks from the provided Technology Radar document. "
        "Focus on Angular ecosystem and other frontend technologies mentioned in the document. For each technology:"
        "\n1. Evaluate current adoption levels in the industry"
        "\n2. Specify whether it's open-source or licensed"
        "\n3. Provide insights on why certain technologies are in their respective quadrants (Adopt, Trial, Assess, Hold)"
        "\n4. Highlight any technologies that should be moved to different quadrants based on current industry trends"
    ),
    context=[{
        "description": "Technology Radar content for frontend evaluation",
        "expected_output": "A detailed analysis of frontend technologies",
        "technology_radar": tech_radar_content
    }],
    expected_output="A comprehensive analysis of frontend technologies with focus on adoption trends, licensing models, and recommendations for quadrant changes.",
    agent=frontend_expert,
)

### Task: Testing Technology Evaluation

In [57]:
testing_eval_task = Task(
    description=(
        "Evaluate testing frameworks, libraries, and tools from the provided Technology Radar document. "
        "Focus on testing technologies across multiple platforms. For each technology:"
        "\n1. Evaluate current adoption levels in the industry"
        "\n2. Specify whether it's open-source or licensed"
        "\n3. Provide insights on why certain technologies are in their respective quadrants (Adopt, Trial, Assess, Hold)"
        "\n4. Highlight any technologies that should be moved to different quadrants based on current industry trends"
    ),
    context=[{
        "description": "Technology Radar content for testing evaluation",
        "expected_output": "A detailed analysis of testing technologies",
        "technology_radar": tech_radar_content
    }],
    expected_output="A comprehensive analysis of testing technologies with focus on adoption trends, licensing models, and recommendations for quadrant changes.",
    agent=testing_expert,
)

### Task: Technology Radar Summary and Recommendations

In [58]:
coordinator_task = Task(
    description=(
        "Review and synthesize the evaluations from the Backend, Frontend, and Testing experts. "
        "Create a comprehensive technology radar summary that highlights:"
        "\n1. Cross-cutting trends and patterns across all domains"
        "\n2. Consolidated list of technologies that should change quadrants"
        "\n3. Licensing model insights across the technology landscape"
        "\n4. Recommendations for technology adoption strategy"
    ),
    context=[],  # Empty list as initial context
    expected_output="A comprehensive technology radar summary with cross-domain insights and strategic recommendations.",
    agent=coordinator,
    async_execution=False,  # Added for task execution flow control
    output_file="technology-radar-synthesis.md",  # Optional output file
    dependencies=[backend_eval_task, frontend_eval_task, testing_eval_task]  # Task dependencies
)

## Creating the Technology Radar Evaluation Crew
 

In [60]:
tech_radar_crew = Crew(
    agents=[backend_expert, frontend_expert, testing_expert, coordinator],
    tasks=[backend_eval_task, frontend_eval_task, testing_eval_task, coordinator_task],
    verbose=True,  # Enable verbose output
    memory=True,  # Enable memory for agent interactions
)

## Running the Technology Radar Analysis Crew

In [ ]:
# Execute the technology radar analysis
# The kickoff method does not need input parameters as we've already provided context to our tasks
tech_radar_result = tech_radar_crew.kickoff()

## Analysis Results and Final Report

In [ ]:
# Print the final coordinated technology radar report
print(tech_radar_result)

## Saving Results to File

In [64]:
# Save the technology radar analysis to a file
with open('updated-technology-radar.md', 'w') as f:
    f.write(str(tech_radar_result))

## Additional Analysis of Updates

In [ ]:
# Display a summary of the main changes suggested by our expert agents
print("Summary of Key Technology Radar Updates:")
print("-" * 50)
print("The multi-agent analysis has completed. The updated Technology Radar includes:")
print("- Evaluation of adoption levels for each technology")
print("- Open-source vs. licensed classification")
print("- Recommended quadrant changes based on current trends")
print("- Cross-domain insights and strategic recommendations")

## Visualization of Technology Radar Changes

In [ ]:
# Optional: If you want to visualize the technology radar changes
# You could install and use a plotting library like matplotlib or plotly

print("To visualize the technology radar, you could:")
print("1. Parse the updated technology radar markdown")
print("2. Extract the technologies by quadrant")
print("3. Plot them using a radar-style visualization")
print("")
print("This would require additional libraries like matplotlib or plotly.")